In [ ]:
library(fcs)
library(tidyverse)
library(ptinpoly)

#' Read a CSV file with gates defined as polygons
#'
#' Currently a gate file has one gate defined as an unclosed polygon (last point
#' differs from first, but it will be closed by the underlying functions.)
#library(devtools)
#' @param filename the name of the file to read, contains a rowname to avoid.
#' @param repair_names logical, if TRUE then repair the names to match the FCS parameter names
#'     " " added before "[" and "_" replaced with "/"
#' @return a tibble of gate vertices
read_gate_csv <- function(filename = "gate1.csv", repair_names = TRUE){
    x <- suppressWarnings(suppressMessages(readr::read_csv(filename[1],
                                     col_types = cols("-", "d", "d"))))
    if (repair_names[1]){
        nm <- colnames(x)
        nm <- gsub("_", "/", nm, fixed = TRUE)
        nm <- gsub("[", " [", nm, fixed = TRUE)
        colnames(x) <- nm
    }
    x
}

#' Read one or more CSV file(s) with gates defined as polygons
#'
#' @param filenames character, one or more filenames
#' @param ... further arguments for \code{read_gate_csv}
#' @return a named list of tibbles of gate vertices
read_gates_csv <- function(filenames = c("gate1.csv", "gate2.csv"), ...){

    xx <- lapply(filenames, read_gate_csv, ...)
    names(xx) <- basename(filenames)
    xx
}



#' Given an Nx2 matrix of coordinates, verify, and possibly modify, that the
#'   polygon is open.
#'
#' @param xy two column matrix of [x,y] defining vertices
#' @return the input polygon, but possibly modified to be open
verify_poly_is_open <- function(xy){
    # make sure the polygon is unclosed
    n <- nrow(xy)
    closed <- identical(xy[1,], xy[n,])
    if (closed) xy <- xy[-n,]
    xy
}


#' Test if points are within the polygon.
#'
#' @seealso ptinpoly::pip2d()
#' @param x FCS data object returned by fcs::FCS()
#' @param gate tibble returned by read_gate()
#' @return vector as returned by ptinpoly::pip2d()
points_in_poly <- function(x = fcs::FCS("AH-135_RS_EEB-Dock_RSG_001.fcs"),
                     gate = read_gate_csv()){
  namesxy <- colnames(gate)
  ix <- namesxy %in% x$parameter_names()
  if (!all(ix)){
    stop("requested parmeters not found in fcs file:", paste(namesxy[!ix], collapse = ", "))
  }
  points <-  x$mget(namesxy)
  poly <- as.matrix(gate[1:2])
  ptinpoly::pip2d(poly, points)
}

#' Given an FCS object and a gate definition, retrieve the gate status of the sorted wells.
#'
#' @param x FCS data object returned by fcs::FCS()
#' @param gate tibble returned by read_gate_csv()
#' @return a tibble with
#' \itemize{
#'   \item{eventnum the event number of the sorted item}
#'   \item{well the A01 style wellname of the sorted item}
#'   \item{gate an integer flag where >= 0 indicates the point is one or within the polygon}
#' }
gate_extract <- function(x = fcs::FCS("AH-135_RS_EEB-Dock_RSG_001.fcs"),
                     gate = read_gate_csv()){
    ix <- x$get_sorted(index = TRUE, wellname = TRUE)
    pip <- points_in_poly(x, gate)

    dplyr::tibble(eventnum = unname(ix),
                  well = names(ix),
                  gate = pip[ix])
}


#' Given an FCS object and one or more gate definitions,
#' retrieve the gate status of the _sorted_ wells for each gate.
#'
#' @param x FCS data object returned by fcs::FCS()
#' @param gate tibble returned by read_gate_csv()
#' @return a tibble with multiple gate IDs
#' \itemize{
#'   \item{eventnum the event number of the sorted item}
#'   \item{well the A01 style wellname of the sorted item}
#'   \item{gateN an integer flag where >= 0 indicates the point is one or within the polygon}
#'   \item{gateM an integer flag where >= 0 indicates the point is one or within the polygon}
#' }
gates_extract <- function(x = fcs::FCS("AH-135_RS_EEB-Dock_RSG_001.fcs"),
                     gates = read_gates_csv()){

    if (is.null(names(gates))) names(gates) <- paste0("gate_",seq_along(gates))
    ix <- x$get_sorted(index = TRUE, wellname = TRUE)
    pp <- lapply(gates, function(g) points_in_poly(x, g))
    names(pp) <- names(gates)

    y <- dplyr::tibble(eventnum = unname(ix), well = names(ix))
    for (nm in names(pp)) y[[nm]] <- pp[[nm]][ix]
    y
}
